# 「行列計算アルゴリズム 第11章 機械学習」用ノートブック

# ■メモ

### 概要
- 第11章「機械学習」で使用するノートブックです。
- 各セルを順に実行することで、本書に記載のとおり実行結果が出力されます。
- 動作確認はJulia 1.10.0 で行っています。

### 使用するスクリプト（.jl）ファイル
- MCA_ml_ridge_regression.jl
- MCA_ml_logistic_regression.jl
- MCA_ml_spectral_clustering.jl
- MCA_ml_pca.jl
- MCA_nlopt_newton.jl（第7章で作成したもの）

### 事前にインストールが必要なパッケージ
- Clustering

# ■プログラム

## 11.1 行列計算の応用としての教師あり機械学習

### 11.1.1 最小二乗問題を利用したリッジ回帰

In [ ]:
include("MCA_ml_ridge_regression.jl");

In [ ]:
using Random
Random.seed!(1234)
ntrain = 100; ntest = 100; n = ntrain + ntest; m = 4
wast = [5, -4, 3, -2, 1]
X = randn(n,m); Y = [ones(n) X] * wast
X = X + 0.5 * randn(n,m)
Xtrain = X[1:ntrain,:]; Ytrain = Y[1:ntrain]
Xtest = X[ntrain+1:n,:]; Ytest = Y[ntrain+1:n];

In [ ]:
lambda = 1
w = MCA_ml_ridge_regression(Xtrain,Ytrain,lambda)
Ypred = [ones(ntest) Xtest] * w;

In [ ]:
err = norm(Ypred-Ytest) / norm(Ytest)
println("err = $err")

### 11.1.2 非線形最適化問題を利用したロジスティック回帰

In [ ]:
# 第7章で作成した MCA_nlopt_newton.jl を読み込む
dir = readdir("../", join=true)
include(dir[7]*"/MCA_nlopt_newton.jl");

In [ ]:
include("MCA_ml_logistic_regression.jl");

In [ ]:
using Random
Random.seed!(1234)
n1 = 200; n2 = 200; n = n1 + n2; m = 2
X1 = randn(n1,m) .+ [1, 1]'; X2 = randn(n2,m) .+ [-1, -1]'
Xtrain = [X1; X2]; Ytrain = [ones(n1); zeros(n2)]
X1 = randn(n1,m) .+ [1, 1]'; X2 = randn(n2,m) .+ [-1, -1]'
Xtest = [X1; X2]; Ytest = [ones(n1); zeros(n2)];

In [ ]:
w = zeros(m+1)
w, ~ = MCA_ml_logistic_regression(Xtrain,Ytrain,w)
Ypred = round.(sigmoid.([ones(n) Xtest] * w));

In [ ]:
accuracy = sum(Ytest .== Ypred) / n
println("accuracy = $accuracy")

## 11.2 行列計算の応用としての教師なし機械学習

### 11.2.1 固有値問題を利用したスペクトラルクラスタリング

In [ ]:
include("MCA_ml_spectral_clustering.jl");

In [ ]:
using Random
Random.seed!(1234)
n = 500; r = round.(rand(n)) .+ 1
theta = range(0,2*pi,n)
X = [r .* cos.(theta) r .* sin.(theta)] .+ 0.1 * randn(n,2)
yast = r .== 1;

In [ ]:
y = MCA_ml_spectral_clustering(X,2);

In [ ]:
using Clustering
nmi = mutualinfo(yast,y)
println("nmi = $nmi")

### 11.2.2 特異値分解を利用した主成分分析

In [ ]:
include("MCA_ml_pca.jl");

In [ ]:
using Random
Random.seed!(1234)
n = 100; m = 3
x = randn(n); X = [x 2*x 3*x] + 1 * randn(n,m);

In [ ]:
B, Y = MCA_ml_pca(X,2)
println("B ="); display(B)

In [ ]:
using Random
Random.seed!(1234)
ntrain = 100; ntest = 100; n = ntrain + ntest; m = 10; ell = 4
wast = randn(m+1)
X = randn(n,ell) * randn(ell,m); Y = [ones(n) X] * wast
X = X + 0.5 * randn(n,m)
Xtrain = X[1:ntrain,:]; Ytrain = Y[1:ntrain]
Xtest = X[ntrain+1:n,:]; Ytest = Y[ntrain+1:n];

In [ ]:
lambda = 1
# （元データに対する）リッジ回帰
w = MCA_ml_ridge_regression(Xtrain,Ytrain,lambda)
Ypred_Org = [ones(ntest) Xtest] * w
# PCA + リッジ回帰
B, ~ = MCA_ml_pca(Xtrain,ell)
w = MCA_ml_ridge_regression(Xtrain*B,Ytrain,lambda)
Ypred_PCA = [ones(ntest) Xtest*B] * w;

In [ ]:
err_Org = norm(Ypred_Org-Ytest) / norm(Ytest)
err_PCA = norm(Ypred_PCA-Ytest) / norm(Ytest)
println("Org: err = $err_Org")
println("PCA: err = $err_PCA")